In [118]:
import logging
import os 
import time
import re
import mysql.connector
import pickle
import os, sys
parent_dir = os.path.dirname(os.getcwd())
#os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
sys.path.append(parent_dir)

from db_func import query_from_db

start = time.time()
#DIR_PATH = os.path.dirname(os.path.realpath(__file__))

#FORMAT = '%(asctime)s %(levelname)s: %(message)s'
#logging.basicConfig(level=logging.INFO, filename=os.path.join(DIR_PATH, 'sent_splitter.log'), filemode='a', format=FORMAT)
raw_df = query_from_db("SELECT news_id, news FROM news_contents WHERE processed_status = 0 ORDER BY RAND() LIMIT 10;")

def remove_header(author_header, header, sent):
    if not re.search(author_header, sent):
        return header, sent
    else:
        header_end = re.search(author_header, sent).end()
        return remove_header(author_header, header + sent[:header_end],sent[header_end:])
    

def insert_sents(raw_df):
    for index, (news_id, content) in raw_df.iterrows():
        #sql_status = "UPDATE news_contents SET processed_status = 1  WHERE news_id = %s"
        #val_status = (news_id, )
        #insert_process(sql_status, val_status)
        author_header = r'^((（.+?）)|(\(.+?\))|(【.+?】)|(〔.+?〕)|(\[.+?\])|(［.+?］))\s*'
        content_footer = r'^更多.*?報導：?$|^更多新聞推薦|^【更多新聞】|^延伸閱讀：|^【延伸閱讀】|^超人氣$|^看更多.*?文章|^更多匯流新聞網報導：|^原始連結|^更多\w+內容：|^《TVBS》提醒您：|^※|^（延伸閱讀：|^相關影音：|^責任編輯：|^☆|^更多\w+相關新聞|►'
        sent_list = []
        author_list = []
        for index, sent in enumerate(re.split(r'\n', content)):
            if re.search(content_footer, sent):
                break
            if index == 0 and len(sent) < 20 and not re.search('。', sent):
                author_list.append(sent)
                continue
            if index <= 1 and re.search(author_header, sent):
                ori_sent = sent
                header, sent = remove_header(author_header, '',  ori_sent)
                author_list.append(header)
            if len(sent) > 1:
                sent_list.append(sent)
        for sent_id, sent in enumerate(sent_list):
            #sent_cursor = mydb.cursor()
            try:
                print((news_id, sent_id + 1, sent))
                #sent_cursor.execute("INSERT INTO news_sents (news_id, sent_index, sent) VALUES (%s, %s, %s)", (news_id, sent_id + 1, sent))
                
            except Exception as e:
                #logging.error('Error: {}\nSentence ID: {}'.format(e, sent_id))
                #mydb.rollback()
                print("Error: ", e)
            else:
                #mydb.commit()
                #sql_success = "UPDATE news_contents SET processed_success = 1  WHERE news_id = %s"
                #val_success = (news_id, )
                #insert_process(sql_success, val_success)
                print()

            #sent_cursor.close()

insert_sents(raw_df)
#mydb.close()
#logging.info('Finished predict {} example in {} seconds'.format(len(raw_df), time.time() - start))

(9427, 1, '針對美國總統拜登上任，中國外交部發言人華春瑩今天說，過去幾年川普政府在中美關係「埋了太多的雷需要排除」，需要治癒恢復及相互尊重，中方希望拜登能在國家治理取得成功。')

(9427, 2, '華春瑩還說，相信在雙方共同努力下，中美關係中「善良的天使能夠戰勝邪惡的力量」；但她對中國制裁蓬佩奧等28名川普政府成員則聲稱，中方的制裁「完全正當和必要」，是對這些人在涉華問題上「嚴重侵犯中國主權、安全、發展利益的錯誤行徑作出的回應」。')

(9427, 3, '中國外交部下午舉行例行記者會，拜登（Joe Biden）上任後的美中關係成為現場關注焦點。澎湃新聞報導，華春瑩在回答相關問題時，作上述表示。')

(9427, 4, '華春瑩說，中方對拜登宣誓就職表示祝賀，也希望拜登在國家治理當中能取得成功。過去一段時間特別不容易，經過那些非常困難和不平凡的時光之後，中美兩國人民都值得擁有一個更加美好的未來。')

(9427, 5, '她聲稱，過去4年來，美國「少數反華政客出於一己私利，撒了太多的謊，煽動了太多的仇恨和分裂，中國和美國人民都深受其害」。拜登在就職演說中提到，美國有「太多需要治癒、太多需要恢復」，對於中美關係也是如此。')

(9427, 6, '華春瑩宣稱，過去幾年中，川普（Donald Trump）政府、特別是時任國務卿蓬佩奧（Mike Pompeo）在中美關係中「埋了太多的雷，需要排除；燒了太多的橋，需要重建；毀了太多的路，需要修復」。')

(9427, 7, '她又說，中美雙方都需要「拿出勇氣，展現智慧，彼此傾聽、正視和相互尊重」，這是中美兩個大國「應有的擔當，也是國際社會的期待」。')

(9427, 8, '此外，華春瑩並針對中國一舉制裁蓬佩奧等28名川普政府成員聲稱，中方的制裁「完全正當和必要」，是對這些人在涉華問題上「嚴重侵犯中國主權、安全、發展利益的錯誤行徑作出的回應」，充分展示了中國政府「捍衛國家利益」的堅定決心。')

(9427, 9, '她宣稱，過去幾年美國「一些反華政客出於一己政治私利和對華偏見、仇恨，策劃以及推動實施一系列瘋狂行徑，嚴重干涉中國內政，損害中國利益，傷害中國人民感情，也嚴重破壞了中美關係」。中方已多次指出，這些反華政客「必將為其瘋狂行徑付出代價」。（編輯：繆宗翰）1100121')

(121917

In [119]:
parent_dir

'/Users/gary/interview/chinese_nlp'

In [111]:
if re.search(r'(^)(（.+?）)|(\(.+?\))|(【.+?】)|(〔.+?〕)\s*','路透社31日報導，包括俄羅斯反對派領袖納瓦尼(Alexei Navalny)'):
    print(1)
    print(re.search(r'(^)(（.+?）)|(\(.+?\))|(【.+?】)|(〔.+?〕)\s*','路透社31日報導，包括俄羅斯反對派領袖納瓦尼(Alexei Navalny)'))
    #print(re.search(r'（.+）\W*','（中央社倫敦18日綜合外電報導）帛琉總統當選人惠恕仁即將於21日宣誓就職，他誓言要對抗中國在太平洋的「霸凌」，並說帛琉將與「真正的朋友」美國及台灣結盟。').end())

1
<_sre.SRE_Match object; span=(22, 38), match='(Alexei Navalny)'>
